In [1]:
import os
import sys
import pathlib
import click
import yaml
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import csv


In [2]:
""" trainer.py """
import tensorflow as tf
from tensorflow.keras.utils import Progbar
import tensorflow.keras as K
from model.dataset import Dataset
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer
from code_jupyter import nnfp_functional
from model.fp.NTxent_loss_single_gpu import NTxentLoss
from model.fp.online_triplet_loss import OnlineTripletLoss
from model.fp.lamb_optimizer import LAMB
from model.utils.experiment_helper import ExperimentHelper
from model.utils.mini_search_subroutines import mini_search_eval

In [3]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    #m_fp = get_fingerprinter(cfg, trainable=False)
    #m_fp = nnfp_functional.get_fingerprinting(input1)
    return m_pre, m_specaug#, m_fp

In [4]:
tr_loss = K.metrics.Mean(name='train_loss')

@tf.function
def train_step(X, m_pre, m_specaug, loss_obj, opt):
    """ Train step """
    # X: (Xa, Xp)
    # Xa: anchors or originals, s.t. [xa_0, xa_1,...]
    # Xp: augmented replicas, s.t. [xp_0, xp_1] with xp_n = rand_aug(xa_n).
    n_anchors = len(X[0])
    print("aqui1")
    X = tf.concat(X, axis=0)
    feat = m_specaug(m_pre(X))  # (nA+nP, F, T, 1)
    #m_fp = nnfp_functional.get_fingerprinting(X)
    #m_fp.trainable = True
    with tf.GradientTape() as t:
        print("aqui2")
        #feat = tf.reshape(feat, shape=[feat.shape[0], 128, -1])
        print(f"feat={feat}")
        m_fp.trainable = True
        m_fp= nnfp_functional.get_fingerprinting(feat)
        
        print("aqui3")
        emb = m_fp(feat)  # (BSZ, Dim)
        print(emb)
        print("aqui4")
        
        loss, sim_mtx, _ = loss_obj.compute_loss(emb[:n_anchors, :], emb[n_anchors:, :]) # {emb_org, emb_rep}
    g = t.gradient(loss, m_fp.trainable_variables)
    m_fp.summary()

    opt.apply_gradients(zip(g, m_fp.trainable_variables))
    
    avg_loss = tr_loss(loss) # To tensorboard.
    
    return avg_loss, sim_mtx # avg_loss: average within the current epoch

In [4]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg


def update_config(cfg, key1: str, key2: str, val):
    cfg[key1][key2] = val
    return cfg


def print_config(cfg):
    os.system("")
    print('\033[36m' + yaml.dump(cfg, indent=4, width=120, sort_keys=False) +
          '\033[0m')
    return

In [6]:
checkpoint_name:str = "CHECKPOINT_BSZ_120"#"CHECKPOINT"   # string
checkpoint_index:int = None  # int
config:str = "default"       # string 'default'
#index_type:str = 'IVFPQ'  # {'L2', 'IVF', 'IVFPQ', " + "'IVFPQ-RR', 'IVFPQ-ONDISK', HNSW'}"
#test_seq_len:str =  '11'   # string '1 3 5 9 11 19' segundos 
#test_ids:str = "icassp"      # string 'icassp'
#nogpu:bool = False         # False or True
max_epoch:int = 1     # int

In [7]:
cfg = load_config(config)

# Dataloader
dataset = Dataset(cfg)

cli: Configuration from ./config/default.yaml


In [ ]:
# Build models.
#m_pre, m_specaug, m_fp = build_fp(cfg)
m_pre, m_specaug = build_fp(cfg)
#m_fp = nnfp_functional.get_fingerprinting(input1)

In [8]:
# Learning schedule
total_nsteps = cfg['TRAIN']['MAX_EPOCH'] * len(dataset.get_train_ds())
if cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS':
    lr_schedule = tf.keras.experimental.CosineDecay(
        initial_learning_rate=float(cfg['TRAIN']['LR']),
        decay_steps=total_nsteps,
        alpha=1e-06)
elif cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS-RESTART':
    lr_schedule = tf.keras.experimental.CosineDecayRestarts(
        initial_learning_rate=float(cfg['TRAIN']['LR']),
        first_decay_steps=int(total_nsteps * 0.1),
        num_periods=0.5,
        alpha=2e-06)
else:
    lr_schedule = float(cfg['TRAIN']['LR'])

# Optimizer
if cfg['TRAIN']['OPTIMIZER'].upper() == 'LAMB':
    opt = LAMB(learning_rate=lr_schedule)
elif cfg['TRAIN']['OPTIMIZER'].upper() == 'ADAM':
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
else:
    raise NotImplementedError(cfg['TRAIN']['OPTIMIZER'])

# Loss objects
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])
else:
    raise NotImplementedError(cfg['LOSS']['LOSS_MODE'])

In [ ]:
ep = 0

# Train
""" Parallelism to speed up preprocessing.............. """
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])

enq = tf.keras.utils.OrderedEnqueuer(
    train_ds, use_multiprocessing=True, shuffle=train_ds.shuffle)
enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
            max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])
i = 0
while i < len(enq.sequence):
    X = next(enq.get()) # X: Tuple(Xa, Xp)
    print(f"mini-batch {i} of {len(enq.sequence)}")

    #X = tf.concat(X, axis=0)
    #m_fp = nnfp_functional.get_fingerprinting(X)
    print("aqui-fora")
    avg_loss, sim_mtx = train_step(X, m_pre, m_specaug, loss_obj_train, opt=opt)
    #print(f"avg_loss {avg_loss}")

    i += 1
enq.stop()
""" End of Parallelism................................. """

Rascunhos:

In [1]:
import os
import sys
import yaml
import tensorflow as tf

2024-05-11 22:02:26.093145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
""" trainer.py """
import tensorflow as tf
from tensorflow.keras.utils import Progbar
import tensorflow.keras as K
from model.dataset import Dataset
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer
from code_jupyter import nnfp_functional
from model.fp.NTxent_loss_single_gpu import NTxentLoss
from model.fp.online_triplet_loss import OnlineTripletLoss
from model.fp.lamb_optimizer import LAMB
from model.utils.experiment_helper import ExperimentHelper
from model.utils.mini_search_subroutines import mini_search_eval

from tensorflow.keras import Input

In [6]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg


def update_config(cfg, key1: str, key2: str, val):
    cfg[key1][key2] = val
    return cfg


def print_config(cfg):
    os.system("")
    print('\033[36m' + yaml.dump(cfg, indent=4, width=120, sort_keys=False) +
          '\033[0m')
    return

In [7]:
config = "default"
cfg = load_config(config)

dataset = Dataset(cfg)

cli: Configuration from ./config/default.yaml


In [5]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    return m_pre, m_specaug

In [6]:
# Learning schedule
total_nsteps = cfg['TRAIN']['MAX_EPOCH'] * len(dataset.get_train_ds())
if cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS':
    lr_schedule = tf.keras.experimental.CosineDecay(
        initial_learning_rate=float(cfg['TRAIN']['LR']),
        decay_steps=total_nsteps,
        alpha=1e-06)
elif cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS-RESTART':
    lr_schedule = tf.keras.experimental.CosineDecayRestarts(
        initial_learning_rate=float(cfg['TRAIN']['LR']),
        first_decay_steps=int(total_nsteps * 0.1),
        num_periods=0.5,
        alpha=2e-06)
else:
    lr_schedule = float(cfg['TRAIN']['LR'])

# Optimizer
if cfg['TRAIN']['OPTIMIZER'].upper() == 'LAMB':
    opt = LAMB(learning_rate=lr_schedule)
elif cfg['TRAIN']['OPTIMIZER'].upper() == 'ADAM':
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
else:
    raise NotImplementedError(cfg['TRAIN']['OPTIMIZER'])

# Loss objects
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])
else:
    raise NotImplementedError(cfg['LOSS']['LOSS_MODE'])

In [7]:
ep = 0

# Train
train_ds = dataset.get_train_ds(0)

enq = tf.keras.utils.OrderedEnqueuer(train_ds, use_multiprocessing=True, shuffle=train_ds.shuffle)

enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'], max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])

In [8]:
train_ds = dataset.get_train_ds(0)

In [9]:
m_pre, m_specaug = build_fp(cfg)

In [10]:
x_tensor = tf.convert_to_tensor(train_ds)

2024-05-11 21:54:12.614733: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 37236480000 exceeds 10% of free system memory.


ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [ ]:
feat = m_specaug(m_pre(x_tensor))  # (nA+nP, F, T, 1)

In [ ]:
model.compile(optimizer=opt, loss=loss_obj_train)

In [ ]:
history = model.fit(train_ds, epochs=cfg['TRAIN']['MAX_EPOCH'])

In [ ]:
i = 0
m_pre, m_specaug = build_fp(cfg)


while i < len(enq.sequence):
    X = next(enq.get()) # X: Tuple(Xa, Xp)
    print(f"mini-batch {i} of {len(enq.sequence)}")

    X = tf.concat(X, axis=0) # X: (Xa, Xp)

    feat = m_specaug(m_pre(X))  # (nA+nP, F, T, 1)

    m_fp= nnfp_functional.get_fingerprinting(feat)


    m_fp = nnfp_functional.get_fingerprinting()
    #avg_loss, sim_mtx = train_step(X, m_pre, m_specaug, loss_obj_train, opt=opt)
    i += 1
enq.stop()

In [ ]:
tr_loss = K.metrics.Mean(name='train_loss')

@tf.function
def train_step(X, m_pre, m_specaug, loss_obj, opt):
    """ Train step """
    
    n_anchors = len(X[0])
    print("aqui1")
    
    feat = m_specaug(m_pre(X))  # (nA+nP, F, T, 1)
    with tf.GradientTape() as t:
        print("aqui2")

        print(f"feat={feat}")
        m_fp.trainable = True
        m_fp= nnfp_functional.get_fingerprinting(feat)
        
        print("aqui3")
        emb = m_fp(feat)  # (BSZ, Dim)
        print(emb)
        print("aqui4")
        
        loss, sim_mtx, _ = loss_obj.compute_loss(emb[:n_anchors, :], emb[n_anchors:, :]) # {emb_org, emb_rep}
    g = t.gradient(loss, m_fp.trainable_variables)
    m_fp.summary()

    opt.apply_gradients(zip(g, m_fp.trainable_variables))
    
    avg_loss = tr_loss(loss) # To tensorboard.
    
    return avg_loss, sim_mtx # avg_loss: average within the current epoch

In [ ]:
X = tf.concat(X, axis=0)
feat = m_specaug(m_pre(X))

In [ ]:
# Construindo os modelos
input_layer = Input(shape=(256, 32, 1))
m_pre, m_specaug = build_fp(cfg)
m_fp = nnfp_functional.get_fingerprinting(input_layer)

# Combinando os modelos em um único modelo
input_layer = Input(shape=(256, 32, 1))

x = m_pre(input_layer)
x = m_specaug(x)
model = m_fp(x)


# Compilando o modelo
#total_nsteps
#loss
#lr


model.compile(optimizer=opt, loss=loss_obj_train)

# Dataloader
dataset = Dataset(cfg)

# Train
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])
history = model.fit(train_ds, epochs=cfg['TRAIN']['MAX_EPOCH'])


In [10]:
from tensorflow.keras.metrics import Mean
from tensorflow.keras import Input

In [ ]:
# Definindo a loss média para o treinamento
tr_loss = Mean(name='train_loss')

@tf.function
def train_step(X, m_pre, m_specaug, m_fp, loss_obj, opt):
    """Passo de treinamento"""
    # Concatenando os inputs
    X_concatenado = tf.concat(X, axis=0)
    # Aplicando as camadas pré-processamento e de aumento espec-aug
    feat = m_specaug(m_pre(X_concatenado))
    
    # Calculando as embeddings
    emb = m_fp(feat)  
    
    # Calculando a loss
    n_anchors = len(X[0])
    loss, _, _ = loss_obj.compute_loss(emb[:n_anchors, :], emb[n_anchors:, :])
    
    # Calculando os gradientes e otimizando
    gradients = opt.gradient(loss, m_fp.trainable_variables)
    opt.apply_gradients(zip(gradients, m_fp.trainable_variables))
    
    # Atualizando a loss média
    avg_loss = tr_loss(loss)
    
    return avg_loss

def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg


def update_config(cfg, key1: str, key2: str, val):
    cfg[key1][key2] = val
    return cfg


def print_config(cfg):
    os.system("")
    print('\033[36m' + yaml.dump(cfg, indent=4, width=120, sort_keys=False) +
          '\033[0m')
    return

# Carregando a configuração
config = "default"
cfg = load_config(config)

# Construindo os modelos
m_pre, m_specaug = build_fp(cfg)
m_fp = nnfp_functional.get_fingerprinting(Input(shape=(256, 32, 1)))



checkpoint_name:str = "CHECKPOINT_BSZ_120"#"CHECKPOINT"   # string
checkpoint_index:int = None  # int
config:str = "default"       # string 'default'
max_epoch:int = 1     # int


# Loop de treinamento
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])
for i, X in enumerate(train_ds):
    print(f"mini-batch {i} of {len(train_ds)}")
    avg_loss = train_step(X, m_pre, m_specaug, m_fp, loss_obj_train, opt)
    print(f"Average loss for mini-batch {i}: {avg_loss}")

In [13]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass == False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.))
    m_fp = nnfp_functional.get_fingerprinting(cfg, trainable=False)
    return m_pre, m_specaug, m_fp


def trainer(cfg, checkpoint_name):
    # Dataloader
    dataset = Dataset(cfg)

    # Build models
    m_pre, m_specaug, m_fp = build_fp(cfg)

    # Learning schedule
    total_nsteps = cfg['TRAIN']['MAX_EPOCH'] * len(dataset.get_train_ds())
    if cfg['TRAIN']['LR_SCHEDULE'].upper() == 'COS':
        lr_schedule = CosineDecay(
            initial_learning_rate=float(cfg['TRAIN']['LR']),
            decay_steps=total_nsteps,
            alpha=1e-06)
    else:
        lr_schedule = float(cfg['TRAIN']['LR'])

    # Optimizer
    if cfg['TRAIN']['OPTIMIZER'].upper() == 'LAMB':
        opt = LAMB(learning_rate=lr_schedule)
    elif cfg['TRAIN']['OPTIMIZER'].upper() == 'ADAM':
        opt = Adam(learning_rate=lr_schedule)
    else:
        raise NotImplementedError(cfg['TRAIN']['OPTIMIZER'])

    # Compile the model
    m_fp.compile(optimizer=opt, loss='sparse_categorical_crossentropy')  # Adjust loss as needed

    # Experiment helper
    helper = ExperimentHelper(checkpoint_name=checkpoint_name, optimizer=opt, model_to_checkpoint=m_fp, cfg=cfg)

    # Training loop
    history = m_fp.fit(dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P']), epochs=cfg['TRAIN']['MAX_EPOCH'])
    
    helper.save_checkpoint()  # Save checkpoint after training


In [14]:
trainer(cfg, '123-teste')

TypeError: get_fingerprinting() got an unexpected keyword argument 'trainable'